## Packages

In [ ]:

import os
from cognite.client import CogniteClient 
from cognite.client.config import ClientConfig
from cognite.client.credentials import OAuthInteractive, OAuthClientCredentials
import pandas as pd

from datetime import date
import json

## Variables

In [ ]:
TENANT_ID = "3b7e4170-8348-4aa4-bfae-06a3e1867469"
CDF_CLUSTER = "api"
CLIENT_NAME = "akerbp"
CLIENT_ID = "779f2b3b-b599-401a-96aa-48bd29132a27"  #Cognite API User access- app registration
COGNITE_PROJECT = "akerbp-dev"
SCOPES = [f"https://{CDF_CLUSTER}.cognitedata.com/.default"]

AUTHORITY_HOST_URI = "https://login.microsoftonline.com"
AUTHORITY_URI = AUTHORITY_HOST_URI + "/" + TENANT_ID
PORT = 53000

## Create CDF Client

In [ ]:
creds = OAuthInteractive(client_id=CLIENT_ID, authority_url=AUTHORITY_URI, scopes=SCOPES) #user interactive
#creds = OAuthClientCredentials(token_url=AUTHORITY_URI + "/oauth2/v2.0/token", client_id="0cadc9bc-c728-4d02-9850-0e3b7a81c88c", client_secret=os.environ["CLIENT_SECRET"], scopes=SCOPES) #non-interactive (app based)

In [ ]:
client_cnf = ClientConfig(client_name=CLIENT_NAME, 
                   base_url=f"https://{CDF_CLUSTER}.cognitedata.com", 
                   project=COGNITE_PROJECT, credentials=creds)
client = CogniteClient(client_cnf)

In [ ]:
status = client.iam.token.inspect() #verify your client token and status
print(status)
if "projects" not in vars(status):
    raise Exception("Token Error!")

## Read CDF Resources

## Timeseries

In [ ]:
time_series_out_list = []
time_series_out_list = client.time_series.list(limit=100)

In [ ]:
unit_df = pd.Series(
    index=[item.unit if item.unit else "Not set" for item in time_series_out_list],
    data=1,
    name="unit"
).groupby(level=0).count()
unit_df

In [ ]:
entry = {
    "date": date.today().strftime(format="%d/%m/%Y"),
    "units": [dict(name = idx, count = val) for idx, val in unit_df.items()]
}
with open(f"./result-{COGNITE_PROJECT}.json", "w") as file:
    json.dump(entry, file)

## 3D models

In [ ]:
three_d_model_list = client.three_d.models.list(limit=-1)
model_revision_list=list()
for three_d_model in three_d_model_list:
    print(f"{three_d_model.name}")

In [ ]:
three_d_model_revision_list = []
three_d_model_revision_list = client.three_d.revisions.list(model_id=6050462298938790, published=False, limit=100)


## GIS

In [ ]:
Feature_types = client.geospatial.list_feature_types()
Feature_types

In [ ]:
feature_aseest_polygons = client.geospatial.list_features(feature_type_external_id="arcgis_asset_polygons_ft", limit=-1)
feature_flowlines = client.geospatial.list_features(feature_type_external_id="arcgis_flowlines_ft", limit=-1)

In [ ]:
feature_aseest_polygons.to_geopandas(geometry="geometry4326").plot()

In [ ]:
feature_flowlines.to_geopandas(geometry="geometry4326").plot()

## RAW

In [ ]:
db_list = client.raw.databases.list(limit=-1)
db_list

In [ ]:

table_list = client.raw.tables.list("EC", limit=5)
table_list

## Assets

In [ ]:
asset_list = client.assets.list(limit=10)
asset_list

## Functions

In [ ]:
functions_list = client.functions.list()
functions_list